# 🔍 Análise de Dados da Camada Gold com SQL em Python

Neste notebook, vamos carregar os dados do bucket S3 usando `pandas` e realizar análises utilizando SQL com `duckdb`. Serão realizadas operações de seleção, filtros, agrupamentos e joins sobre a camada gold particionada por `anomesdia`.

In [11]:
import pandas as pd
import duckdb

# DuckDB – Banco de Dados Analítico Embutido

O **DuckDB** é um **banco de dados analítico embutido (embedded analytical database)**, projetado para ser leve, rápido e fácil de usar diretamente em aplicações locais — especialmente em ambientes **Python, R, SQL CLI** ou notebooks (como Jupyter).

---

## 🦆 O que é o DuckDB?

O DuckDB é como um **SQLite para análise de dados**:

- ✅ Roda localmente, **sem servidor**
- ✅ Usa **arquitetura columnar** (ideal para leitura e agregação de grandes volumes de dados)
- ✅ É otimizado para consultas **OLAP** (analíticas), diferente de bancos voltados para transações (**OLTP**)

---

## 🔍 Principais Características

| Característica                | Detalhes                                                                 |
|------------------------------|--------------------------------------------------------------------------|
| ⚙️ **Embutido (Embedded)**   | Roda dentro do seu script/app, como o SQLite                             |
| 🧠 **SQL padrão**             | Sintaxe SQL moderna e compatível                                         |
| 💾 **Columnar**               | Armazena e processa dados em colunas, como o formato Parquet             |
| 🧮 **Desempenho analítico**   | Excelente para `SELECT`, `JOIN`, `GROUP BY`, `FILTER` em grandes datasets|
| 🔗 **Integrações fáceis**     | Funciona com CSV, Parquet, Pandas, Arrow, SQLite, entre outros           |
| 📦 **Sem dependências externas** | Um único binário leve, sem necessidade de instalação complexa         |
| 💻 **Plataformas**            | Compatível com Linux, macOS, Windows e Jupyter Notebooks                 |


# 🧩 Tipos de Comandos SQL (Categorias)

| Tipo | Nome                     | Função Principal                         | Exemplos                              |
|------|--------------------------|------------------------------------------|----------------------------------------|
| DDL  | Data Definition Language | Definição da estrutura do banco de dados | `CREATE`, `ALTER`, `DROP`, `TRUNCATE` |
| DML  | Data Manipulation Language | Manipulação dos dados nas tabelas      | `SELECT`, `INSERT`, `UPDATE`, `DELETE` |
| DCL  | Data Control Language     | Controle de permissões e segurança      | `GRANT`, `REVOKE`                      |
| TCL  | Transaction Control Language | Controle de transações (confirmação e reversão) | `COMMIT`, `ROLLBACK`, `SAVEPOINT` |

---

## 🔧 1. DDL – *Data Definition Language*

Usada para **criar ou modificar a estrutura** do banco (tabelas, colunas, índices, etc).

| Comando        | Função                                             |
|----------------|----------------------------------------------------|
| `CREATE TABLE` | Cria uma nova tabela                               |
| `ALTER TABLE`  | Altera a estrutura de uma tabela existente         |
| `DROP TABLE`   | Exclui uma tabela                                  |
| `TRUNCATE TABLE` | Remove todos os dados da tabela (sem log)       |

### 🧠 Exemplo:

```sql
CREATE TABLE clientes (
    id INT PRIMARY KEY,
    nome VARCHAR(100),
    email VARCHAR(100)
);


In [ ]:
# Criação da tabela temporária com uma consulta
duckdb.sql("""
    CREATE TEMP TABLE top_clientes AS
    SELECT id_cliente, SUM(valor_total) AS total_gasto
    FROM 'dados/vendas.parquet'
    GROUP BY id_cliente
    ORDER BY total_gasto DESC
    LIMIT 10
""")

# Consulta usando a tabela temporária
df = duckdb.sql("SELECT * FROM top_clientes").df()

## 📦 2. DML – *Data Manipulation Language*

Usada para **inserir, consultar, alterar e excluir dados** nas tabelas.

| Comando     | Função                        |
|-------------|-------------------------------|
| SELECT      | Consulta dados                 |
| INSERT INTO | Insere novos registros         |
| UPDATE      | Atualiza registros existentes  |
| DELETE      | Remove registros               |

### 🧠 Exemplo:

```sql
INSERT INTO clientes (id, nome, email) VALUES (1, 'Ana', 'ana@email.com');

SELECT * FROM clientes;

UPDATE clientes SET nome = 'Ana Silva' WHERE id = 1;

DELETE FROM clientes WHERE id = 1;


## 🔐 3. DCL – *Data Control Language*

Usada para **gerenciar permissões** no banco de dados.

| Comando | Função             |
|---------|--------------------|
| GRANT   | Concede permissões |
| REVOKE  | Revoga permissões  |

### 🧠 Exemplo:

```sql
GRANT SELECT ON clientes TO usuario1;
REVOKE SELECT ON clientes FROM usuario1;


## 🔄 4. TCL – *Transaction Control Language*

Usada para **gerenciar transações**, garantindo atomicidade e consistência.

| Comando   | Função                                                          |
|-----------|-----------------------------------------------------------------|
| COMMIT    | Confirma a transação                                            |
| ROLLBACK  | Cancela a transação                                             |
| SAVEPOINT | Define um ponto de salvamento para possível rollback parcial   |

### 🧠 Exemplo:

```sql
BEGIN;
UPDATE contas SET saldo = saldo - 100 WHERE id = 1;
UPDATE contas SET saldo = saldo + 100 WHERE id = 2;
COMMIT;



## 📝 Conclusão

| Categoria | DuckDB Suporta? | Observação                         |
|-----------|------------------|------------------------------------|
| DDL       | ✅ Sim           | Totalmente suportado              |
| DML       | ✅ Sim           | Totalmente suportado              |
| DCL       | ❌ Não           | DuckDB não gerencia usuários      |
| TCL       | ⚠️ Parcial       | Suporta transações simples        |


## 💡 Dica Extra: Ordem natural de execução

Em um sistema real, a sequência geralmente é:

1. **Criar as tabelas (DDL)**
2. **Inserir dados (DML)**
3. **Gerenciar permissões (DCL)**
4. **Garantir integridade via transações (TCL)**


---

# ✅ Exemplo: Lendo um arquivo Parquet no DuckDB e usando SQL

## 📁 Suponha que você tem um arquivo:

```bash
dados/clientes.parquet



## 💡 Leitura e consulta direta no Python:

```python

import duckdb

### Conecta ao banco (em memória ou para arquivo .duckdb)
conn = duckdb.connect()

### Consulta direta no Parquet (sem precisar criar tabela antes)
query = """
SELECT nome, email
FROM 'dados/clientes.parquet'
WHERE nome LIKE 'A%'
"""

resultado = conn.execute(query).fetchdf()
print(resultado) """

---

## 🛠️ Criando uma TABELA a partir do Parquet (DDL + DML)

Se você quiser transformar em uma tabela interna:

```python

## Cria uma tabela a partir do Parquet
conn.execute("""
CREATE TABLE clientes AS
SELECT * FROM 'dados/clientes.parquet'
""")

# Agora você pode fazer DML
conn.execute("SELECT COUNT(*) FROM clientes").fetchall()


---

## 📌 Dicas úteis

| Tarefa                            | Comando                                                   |
|----------------------------------|------------------------------------------------------------|
| Listar colunas                   | `PRAGMA table_info('clientes')`                           |
| Ver 5 primeiras linhas           | `SELECT * FROM clientes LIMIT 5`                          |
| Criar Parquet a partir da tabela | `COPY clientes TO 'clientes_novo.parquet' (FORMAT 'parquet')` |



## 📈 Benefícios do DuckDB com Parquet

✅ Leitura lazy e muito rápida  
✅ Sem precisar importar tudo para RAM  
✅ Ideal para análises locais em grandes datasets  
✅ Sintaxe SQL completa para filtrar, agregar, transformar


## 💡 Quando usar o DuckDB?

Use o **DuckDB** quando você precisa de:

- ✅ Processar dados **locais** de forma rápida e analítica
- ✅ Evitar a complexidade de configurar um PostgreSQL, Redshift, BigQuery, etc.
- ✅ Fazer **análises em arquivos Parquet/CSV** sem subir para a nuvem
- ✅ Trabalhar com dados em **Pandas ou Polars**, mas utilizando **SQL puro**

---

## 🆚 Comparação ampliada de bancos para análise

| Banco         | Modelo         | Nuvem        | Gerenciado | Preço estimado            | Uso típico                                 |
|---------------|----------------|--------------|------------|----------------------------|---------------------------------------------|
| **DuckDB**    | Analítico local| ❌ Local      | ❌ Não      | Gratuito (embutido)        | Análises locais, data science, notebooks    |
| **SQLite**    | Transacional   | ❌ Local      | ❌ Não      | Gratuito                   | Aplicações mobile, desktop, protótipos      |
| **PostgreSQL**| OLTP/OLAP      | ✅ Opcional   | ⚠️ Parcial | Gratuito local ou R$~200+/m (em nuvem) | Web apps, BI, aplicações transacionais     |
| **BigQuery**  | Analítico cloud| ✅ Google Cloud | ✅ Sim   | Paga por consulta e armazenamento | Data Lake, Big Data, análises massivas     |
| **Amazon RDS**| OLTP/OLAP      | ✅ AWS        | ✅ Sim      | R$~100 a R$1000+/m          | Bancos relacionais gerenciados na nuvem     |


## 📌 Mão na massa

## Executando UPDATE, DELETE e TRUNCATE

### 🚧 Alterações em arquivos Parquet com DuckDB

DuckDB não permite `UPDATE` ou `DELETE` diretamente em arquivos Parquet.  

Manipular dados em arquivos **Parquet** diretamente com comandos como `UPDATE` e `DELETE` **não é suportado nativamente**, porque o formato Parquet é **imutável** (colunar, otimizado para leitura e não para escrita em nível de linha).

Mas existem estratégias e ferramentas para contornar isso, dependendo do seu stack de tecnologia.

---

🧠 **Conceito: Parquet é _read-optimized_**

- O Parquet **não foi projetado para atualizações linha a linha**.
- Ele é **ótimo para leitura e compressão**, mas para atualizar dados, é necessário **reescrever o arquivo**.

Para isso, vamos seguir os seguintes passos:

1. Crie uma tabela temporária ou persistente a partir do Parquet.
2. Execute os comandos SQL (`UPDATE`, `DELETE`, `TRUNCATE`) nessa tabela.
3. Salve a tabela modificada de volta para um novo arquivo Parquet.

1. 🛠️ Criar tabela temporária a partir de Parquet

In [ ]:
# Prefixos S3 (já montado com s3fs configurado)
GOLD_PREFIX = 's3://aula-data-lake/gold/'

# Lendo os dados Parquet da camada Gold
fato_vendas = pd.read_parquet(GOLD_PREFIX + 'fato_vendas/')
vendas_por_cliente = pd.read_parquet(GOLD_PREFIX + 'vendas_por_cliente/')
vendas_por_tipo = pd.read_parquet(GOLD_PREFIX + 'vendas_por_tipo/')

In [ ]:
duckdb.sql("""
    CREATE TABLE vendas_temp AS 
    SELECT * FROM fato_vendas
""")

In [ ]:

# Monta a query como string Python
query = f"""
    CREATE TABLE vendas_temp AS 
    SELECT * FROM '{GOLD_PREFIX}fato_vendas/*.parquet'
"""

# Executa a query
duckdb.sql(query)

2. ✏️ Exemplo de UPDATE

In [ ]:
duckdb.sql("""
    SELECT DISTINCT status FROM vendas_temp
""")

In [ ]:
duckdb.sql("""
    BEGIN TRANSACTION;
    UPDATE vendas_temp
    SET status = 'Cancelado'
    WHERE status = 'PENDENTE'
""")

In [ ]:
duckdb.sql("""
    ROLLBACK;
""")

In [ ]:
duckdb.sql("""
    BEGIN TRANSACTION;
    UPDATE vendas_temp
    SET status = 'PENDENTE'
    WHERE status = 'Cancelado';
    COMMIT;    
""")

In [ ]:
duckdb.sql("""
    SELECT DISTINCT status FROM vendas_temp 
    --WHERE status = 'Cancelado'
""")

In [ ]:
duckdb.sql("""
    COMMIT;    
""")

3. 🗑️ Exemplo de DELETE

In [ ]:
duckdb.sql("""
    SELECT *
    FROM vendas_temp
    --WHERE preco_unitario < 40
""")

In [ ]:
duckdb.sql("""
    DELETE FROM vendas_temp
    WHERE preco_unitario < 40
""")

4. 🚨 Exemplo de TRUNCATE (equivalente a apagar tudo)

In [ ]:
duckdb.sql("DELETE FROM vendas_temp")
# ou, equivalente:
# duckdb.sql("TRUNCATE TABLE vendas_temp")

💾 Salvar de volta para Parquet após as alterações

In [ ]:
duckdb.sql("""
    COPY vendas_temp TO 'caminho/novo/fato_vendas_limpo.parquet' (FORMAT PARQUET)
""")

In [ ]:
%pip install duckdb

## 1. 🧾 Seleção de Dados
📝 Exibindo as primeiras linhas da tabela de fatos de vendas para inspeção inicial.

In [ ]:
duckdb.sql("SELECT * FROM fato_vendas LIMIT 5")

In [ ]:
duckdb.sql("DESCRIBE fato_vendas")

In [ ]:
duckdb.sql("SELECT * FROM vendas_por_tipo LIMIT 5")

In [ ]:
duckdb.sql("DESCRIBE vendas_por_tipo")

In [ ]:
duckdb.sql("SELECT * FROM vendas_por_tipo LIMIT 5")

In [ ]:
duckdb.sql("DESCRIBE vendas_por_cliente")

📝 Top 5 clientes que mais compraram em valor total.

In [ ]:
duckdb.sql('''
SELECT 
    id_cliente, 
    nome, 
    ROUND(valor_total_comprado) valor_total_comprado
FROM vendas_por_cliente 
ORDER BY valor_total_comprado DESC 
LIMIT 5;''')

Top 5 tipos de produto mais vendidos em valor.

In [ ]:
duckdb.sql('''SELECT nome_tipo, total_vendido FROM vendas_por_tipo ORDER BY total_vendido DESC LIMIT 5;''')

# 2. 🔍 Funções e Operadores no WHERE – DuckDB

A cláusula `WHERE` é usada para **filtrar linhas** com base em condições. Abaixo estão os operadores e funções mais comuns em `WHERE`, com exemplos práticos no DuckDB.

---

## ✅ Operadores de Comparação

| Operador | Descrição                  | Exemplo DuckDB                               |
|----------|----------------------------|----------------------------------------------|
| =        | Igual                      | `WHERE idade = 30`                           |
| != ou <> | Diferente                  | `WHERE cidade <> 'São Paulo'`               |
| >        | Maior que                  | `WHERE salario > 3000`                       |
| <        | Menor que                  | `WHERE idade < 25`                           |
| >=       | Maior ou igual             | `WHERE idade >= 18`                          |
| <=       | Menor ou igual             | `WHERE idade <= 60`                          |

---

## 🧠 Lógicos

| Operador | Descrição             | Exemplo DuckDB                                           |
|----------|------------------------|----------------------------------------------------------|
| AND      | Todas as condições     | `WHERE idade > 18 AND cidade = 'São Paulo'`             |
| OR       | Uma ou outra           | `WHERE idade < 18 OR salario < 2000`                    |
| NOT      | Negação lógica         | `WHERE NOT (cidade = 'São Paulo')`                      |

---

## 🧪 Operadores especiais

### IN

Verifica se um valor está dentro de uma lista:

```sql
SELECT * FROM clientes
WHERE cidade IN ('São Paulo', 'Campinas', 'Santos');


---

## 🔄 BETWEEN

Verifica se um valor está entre dois limites (inclusive):

```sql
SELECT * FROM vendas
WHERE data BETWEEN DATE '2024-01-01' AND DATE '2024-01-31';


---

## 🔤 LIKE (com curingas)

Faz correspondência parcial de strings:

| Padrão | Significado                      |
|--------|----------------------------------|
| `%`    | Qualquer sequência de caracteres |
| `_`    | Um único caractere               |

```sql
SELECT * FROM clientes
WHERE nome LIKE 'A%';  -- Começa com A

SELECT * FROM clientes
WHERE email LIKE '%@gmail.com'; -- Termina com @gmail.com


---

## ❓ IS NULL / IS NOT NULL

Verifica valores nulos (ausentes):

```sql
SELECT * FROM clientes
WHERE telefone IS NULL;

SELECT * FROM clientes
WHERE telefone IS NOT NULL;


---

## 🧮 Funções numéricas e de texto no WHERE

```sql
SELECT * FROM produtos
WHERE LOWER(categoria) = 'eletrônicos';

SELECT * FROM clientes
WHERE LENGTH(nome) > 10;

SELECT * FROM pedidos
WHERE EXTRACT(YEAR FROM data_pedido) = 2024;


Selecionando todas as vendas feitas no dia `20250101` com valor acima de 100 reais.

In [ ]:
duckdb.sql("""
SELECT * 
FROM fato_vendas 
WHERE anomesdia = '20250101' AND valor_total_item > 100
LIMIT 10
""")

Clientes com mais de 3 pedidos realizados.

In [ ]:
duckdb.sql('''SELECT nome, num_pedidos FROM vendas_por_cliente WHERE num_pedidos > 3;''')

Clientes que compraram entre R$100 e R$500 no total.

In [ ]:
duckdb.sql('''SELECT nome, valor_total_comprado FROM vendas_por_cliente WHERE valor_total_comprado BETWEEN 100 AND 500;''')

## 3. 📊 Agregações
📝 Total vendido por dia (anomesdia), ordenando pelo maior faturamento.

In [ ]:
duckdb.sql("""
SELECT anomesdia, ROUND(SUM(valor_total_item), 2) AS faturamento_total
FROM fato_vendas
GROUP BY anomesdia
ORDER BY faturamento_total DESC
LIMIT 10
""")

📊 **Tipos de Agregações em SQL**

🔢 **1. Agregações Numéricas (estatísticas básicas)**

| Função SQL           | Descrição               |
|----------------------|--------------------------|
| `SUM()`              | Soma dos valores         |
| `AVG()`              | Média aritmética         |
| `MIN()`              | Valor mínimo             |
| `MAX()`              | Valor máximo             |
| `COUNT()`            | Contagem de registros    |
| `VAR()` / `VAR_SAMP()`      | Variância               |
| `STDDEV()` / `STDDEV_SAMP()`| Desvio padrão          |


📝 Número total de clientes e o valor médio comprado.

📊 **Exemplos de Agregações Numéricas em SQL usando DuckDB**

| Função SQL                    | Exemplo (DuckDB SQL)                                                         | Tabela               | Explicação                                                                 |
|------------------------------|------------------------------------------------------------------------------|----------------------|----------------------------------------------------------------------------|
| `SUM()`                      | `SELECT SUM(valor_total_item) FROM fato_vendas;`                             | `fato_vendas`        | Soma total de todos os itens vendidos.                                     |
| `AVG()`                      | `SELECT AVG(preco_unitario) FROM fato_vendas;`                               | `fato_vendas`        | Preço médio unitário dos produtos.                                         |
| `MIN()`                      | `SELECT MIN(valor_total_comprado) FROM vendas_por_cliente;`                  | `vendas_por_cliente` | Valor mínimo gasto por um cliente.                                         |
| `MAX()`                      | `SELECT MAX(total_vendido) FROM vendas_por_tipo;`                            | `vendas_por_tipo`    | Valor máximo vendido entre os tipos de produto.                            |
| `COUNT()`                    | `SELECT COUNT(DISTINCT id_pedido) FROM fato_vendas;`                         | `fato_vendas`        | Número de pedidos distintos registrados.                                   |
| `VAR_SAMP()` (ou `VAR_POP()`)| `SELECT VAR_SAMP(quantidade) FROM fato_vendas;`                              | `fato_vendas`        | Variância amostral da quantidade de itens por linha de pedido.             |
| `STDDEV_SAMP()`              | `SELECT STDDEV_SAMP(num_pedidos) FROM vendas_por_cliente;`                   | `vendas_por_cliente` | Desvio padrão do número de pedidos por cliente.                            |


In [ ]:
# Soma total de itens vendidos
duckdb.sql("SELECT SUM(valor_total_item) AS soma_total FROM fato_vendas")

In [ ]:
# Preço médio unitário dos produtos
duckdb.sql("SELECT AVG(preco_unitario) AS preco_medio FROM fato_vendas").df()

In [ ]:
# Valor mínimo gasto por um cliente
duckdb.sql("SELECT MIN(valor_total_comprado) AS valor_minimo FROM vendas_por_cliente")

In [ ]:
# Valor máximo vendido entre os tipos de produto
duckdb.sql("SELECT MAX(total_vendido) AS valor_maximo FROM vendas_por_tipo")

In [ ]:
# Número de pedidos distintos
duckdb.sql("SELECT COUNT(DISTINCT id_pedido) AS pedidos_unicos FROM fato_vendas")

In [ ]:
# Variância amostral da quantidade de itens
duckdb.sql("SELECT VAR_SAMP(quantidade) AS variancia_quantidade FROM fato_vendas")

In [ ]:
# Desvio padrão do número de pedidos por cliente
duckdb.sql("SELECT STDDEV_SAMP(num_pedidos) AS desvio_padrao_pedidos FROM vendas_por_cliente")

In [ ]:
# Variações de variância e desvio padrão: amostral (VAR_SAMP(), STDDEV_SAMP()) e populacional (VAR_POP(), STDDEV_POP()), 
# aplicadas ao campo quantidade da tabela fato_vendas:

duckdb.sql("""
SELECT
    VAR_SAMP(quantidade)     AS variancia_amostral,
    VAR_POP(quantidade)      AS variancia_populacional,
    STDDEV_SAMP(quantidade)  AS desvio_padrao_amostral,
    STDDEV_POP(quantidade)   AS desvio_padrao_populacional
FROM fato_vendas
""")


## 📊 GROUP BY e ORDER BY no SQL

### 🧩 GROUP BY – Agrupamento de Dados

O `GROUP BY` é usado para **agrupar linhas que possuem os mesmos valores** em colunas específicas. Ele é geralmente utilizado com funções de agregação como:

- `SUM()` – soma
- `AVG()` – média
- `COUNT()` – contagem
- `MAX()` – valor máximo
- `MIN()` – valor mínimo

🔍 **Exemplo: Total gasto por cliente**

```sql
SELECT id_cliente, SUM(valor_total) AS total_gasto
FROM vendas
GROUP BY id_cliente;


In [ ]:
duckdb.sql("""
SELECT
    nome_tipo,
    COUNT(*) AS total_registros,
    AVG(quantidade) AS media_quantidade,
    VAR_SAMP(quantidade) AS variancia_amostral,
    VAR_POP(quantidade)  AS variancia_populacional,
    STDDEV_SAMP(quantidade) AS desvio_padrao_amostral,
    STDDEV_POP(quantidade)  AS desvio_padrao_populacional
FROM fato_vendas
GROUP BY nome_tipo
""").df()

---

## ✅ Combinando os dois

É comum combinar `GROUP BY` e `ORDER BY` para **agrupar os dados** e depois **ordenar os resultados agregados**:

```sql
SELECT cidade, COUNT(*) AS num_vendas
FROM vendas
GROUP BY cidade
ORDER BY num_vendas DESC;


### 🔽 Sobre `DESC` e `ASC`

- **ASC (Ascending)** – ordena de forma **crescente** (menor para maior).  
  É o comportamento padrão, mesmo que você não escreva explicitamente.

- **DESC (Descending)** – ordena de forma **decrescente** (maior para menor),  
  muito útil para visualizar os "top resultados".



✅ **Estatísticas agrupadas por tipo de produto (nome_tipo)**

In [ ]:
duckdb.sql("""
SELECT
    nome_tipo,
    COUNT(*) AS total_registros,
    AVG(quantidade) AS media_quantidade,
    VAR_SAMP(quantidade) AS variancia_amostral,
    VAR_POP(quantidade)  AS variancia_populacional,
    STDDEV_SAMP(quantidade) AS desvio_padrao_amostral,
    STDDEV_POP(quantidade)  AS desvio_padrao_populacional
FROM fato_vendas
GROUP BY nome_tipo
ORDER BY total_registros DESC
""").df()

🔎 **O que esse SELECT retorna:**

| Coluna                      | Significado                                                        |
|-----------------------------|---------------------------------------------------------------------|
| `nome_tipo`                 | Categoria do produto                                               |
| `total_registros`           | Quantidade de linhas (itens) daquele tipo                          |
| `media_quantidade`          | Quantidade média vendida por linha para aquele tipo                |
| `variancia_amostral`        | Dispersão amostral da quantidade                                   |
| `variancia_populacional`    | Dispersão populacional da quantidade                               |
| `desvio_padrao_amostral`    | Raiz da variância amostral                                         |
| `desvio_padrao_populacional`| Raiz da variância populacional                                     |


## 🧠 Entendendo o uso do `HAVING` em SQL

### 📌 O que é `HAVING`?

A cláusula `HAVING` é usada para **filtrar os resultados de uma agregação** feita com `GROUP BY`.  
Enquanto o `WHERE` filtra **linhas individuais antes da agregação**, o `HAVING` filtra **grupos após a agregação**.


### 🔄 Diferença entre `WHERE` e `HAVING`

| Cláusula | Quando é aplicada          | Usada com agregações? |
|----------|----------------------------|------------------------|
| `WHERE`  | Antes do `GROUP BY`         | ❌ Não (linhas simples) |
| `HAVING` | Depois do `GROUP BY`        | ✅ Sim                  |



### ✅ Exemplo prático com `HAVING`

```sql
SELECT nome_tipo, AVG(quantidade) AS media
FROM fato_vendas
GROUP BY nome_tipo
HAVING AVG(quantidade) > 5;


🔍 **Explicação:**

- Agrupa os dados por `nome_tipo`.
- Calcula a média da `quantidade` para cada grupo.
- Retorna apenas os grupos onde a média é maior que 5.



💡 **Quando usar `HAVING`?**  
Use `HAVING` quando:

- Você está usando `GROUP BY`.
- Precisa filtrar os grupos com base em funções de agregação (`SUM`, `AVG`, `COUNT`, etc).


🚫 **Exemplo incorreto:**

```sql
SELECT nome_tipo, AVG(quantidade)
FROM fato_vendas
WHERE AVG(quantidade) > 5  -- ❌ ERRO!
GROUP BY nome_tipo;

🛑 **Não é permitido usar funções de agregação em `WHERE`. Use `HAVING`.**

📝 **Resumo:**

- `WHERE`: filtra **linhas** antes da agregação.
- `HAVING`: filtra **grupos** depois da agregação.
- Só use `HAVING` se estiver fazendo `GROUP BY`.

✅ **Exemplo: GROUP BY com HAVING no DuckDB**



In [ ]:
duckdb.sql("""
SELECT
    nome_tipo,
    COUNT(*) AS total_itens,
    AVG(quantidade) AS media_quantidade
FROM fato_vendas
GROUP BY nome_tipo
HAVING AVG(quantidade) > 5
ORDER BY media_quantidade DESC
""").df()

## 4. 🤝 Join entre Fato e Dimensão Cliente

## ✅ Passos para criar tabela dim_cliente

1. Ler a tabela Parquet original  
2. Criar a dimensão com `id_cliente` e `nome` (sem duplicatas)  
3. Salvar como novo arquivo Parquet


In [ ]:
%pip install boto3==1.34.96 botocore==1.34.96 s3fs==2024.3.1 fsspec==2024.3.1

In [ ]:
import duckdb  # Banco de dados analítico embutido que permite executar SQL sobre arquivos locais e S3

import pandas as pd  # Biblioteca para manipulação e análise de dados em DataFrames

import pyarrow as pa  # Apache Arrow: usado para representar dados em memória de forma eficiente (colunar)

import pyarrow.dataset as ds  # Importa o módulo de datasets do PyArrow, que permite escrever dados em múltiplos formatos (como Parquet) com mais controle, incluindo particionamento e uso de sistemas de arquivos como S3

import s3fs  # Biblioteca que permite interagir com buckets S3 como se fossem sistemas de arquivos locais


In [ ]:
DIM_PREFIX = "s3://aula-data-lake/dim/"

# 1. Lê os dados com DuckDB e gera o DataFrame pandas
query = f"""
    SELECT DISTINCT id_cliente, nome as nome_cliente
    FROM vendas_por_cliente
"""

dim_cliente_df = duckdb.sql(query).df()

dim_cliente_df.to_parquet(DIM_PREFIX + 'dim_cliente/', index=False)


In [ ]:
dim_cliente = pd.read_parquet(DIM_PREFIX + 'dim_cliente/')

In [ ]:
duckdb.sql("""
SELECT *
FROM dim_cliente
ORDER BY 1
--LIMIT 10
""")

Análise de clientes que mais compraram.

In [ ]:
duckdb.sql("""
SELECT c.id_cliente, c.nome, c.valor_total_comprado, c.num_pedidos
FROM vendas_por_cliente c
ORDER BY valor_total_comprado DESC
LIMIT 10
""")

## 🔗 O que é um `JOIN` em SQL?

`JOIN` é uma operação em SQL que permite **combinar dados de duas ou mais tabelas** com base em uma condição lógica, normalmente uma chave em comum (como `id_cliente`, `id_produto`, etc).

É uma das ferramentas mais poderosas para análise de dados relacionais, pois permite reunir informações que estão distribuídas em diferentes tabelas.

---

## 🤝 Tipos de `JOIN` mais comuns

![Tipos de JOIN](img/joins.png)

### 1. `INNER JOIN` – Interseção

Retorna apenas os registros que **têm correspondência em ambas as tabelas**.

🔍 **Usado quando** você só quer registros que existem em ambas as tabelas.


In [17]:
print(f'Quantidade de linhas e colunas da tabela fato_vendas')
print(fato_vendas.shape)
print("---------------------------------------------------------------------------------------------------------------")
print(f'Quantidade de linhas e colunas da tabela vendas_por_cliente')
print(vendas_por_cliente.shape)

Quantidade de linhas e colunas da tabela fato_vendas
(300, 19)
---------------------------------------------------------------------------------------------------------------
Quantidade de linhas e colunas da tabela vendas_por_cliente
(51, 4)


In [29]:
# Executa as instruções de DROP e CREATE (sem retorno)
duckdb.sql("""
    DROP TABLE IF EXISTS fato_vendas_temp;
    DROP TABLE IF EXISTS vendas_por_cliente_temp;

    CREATE TABLE vendas_por_cliente_temp AS 
    SELECT * FROM vendas_por_cliente LIMIT 40;
""")

# Agora faz a leitura da tabela e envia para DataFrame
vendas_por_cliente_temp = duckdb.sql("SELECT * FROM vendas_por_cliente_temp").df()


In [30]:
print(f'Quantidade de linhas e colunas da tabela vendas_por_cliente_tmp')
print(vendas_por_cliente_temp.shape)

Quantidade de linhas e colunas da tabela vendas_por_cliente_tmp
(40, 4)


In [ ]:
duckdb.sql('''
SELECT 
    LEFT(CAST(f.anomesdia AS VARCHAR), 6) anomes, 
    c.nome as nome_cliente, 
    ROUND(SUM(f.valor_total_item),2) AS total
FROM fato_vendas f
INNER JOIN dim_cliente c ON f.id_cliente = c.id_cliente
GROUP BY 
    LEFT(CAST(f.anomesdia AS VARCHAR), 6), 
    c.nome
ORDER BY 
    1 DESC,
    2 DESC
LIMIT 10
''')

In [31]:
duckdb.sql('''
SELECT f.anomesdia, c.nome as nome_cliente, f.valor_total_item as vl_total
FROM fato_vendas f
JOIN vendas_por_cliente_temp c ON f.id_cliente = c.id_cliente
''')

┌───────────┬───────────────┬────────────────────┐
│ anomesdia │ nome_cliente  │      vl_total      │
│   int32   │    varchar    │       double       │
├───────────┼───────────────┼────────────────────┤
│  20240620 │ Yuri Teixeira │              32.64 │
│  20240620 │ Yuri Teixeira │             423.35 │
│  20240620 │ Yuri Teixeira │              32.64 │
│  20240620 │ Yuri Teixeira │             423.35 │
│  20240620 │ Yuri Teixeira │              32.64 │
│  20240620 │ Yuri Teixeira │             423.35 │
│  20240624 │ Thiago Castro │            2470.35 │
│  20240624 │ Thiago Castro │             137.48 │
│  20240624 │ Thiago Castro │             111.35 │
│  20240624 │ Thiago Castro │            2470.35 │
│      ·    │      ·        │               ·    │
│      ·    │      ·        │               ·    │
│      ·    │      ·        │               ·    │
│  20250604 │ Levi Correia  │            1674.35 │
│  20250608 │ Brenda Fogaça │ 1135.1100000000001 │
│  20250608 │ Brenda Fogaça │  

---

### 2. `LEFT JOIN` (ou `LEFT OUTER JOIN`) – Todos da esquerda

Retorna **todos os registros da tabela da esquerda** e os correspondentes da direita.  
Onde não houver correspondência, os campos da tabela da direita vêm como `NULL`.

🔍 **Usado quando** você quer manter todos os clientes, mesmo que não tenham vendas.

In [32]:
duckdb.sql('''
SELECT f.anomesdia, c.nome as nome_cliente, f.valor_total_item as vl_total
FROM fato_vendas f
LEFT JOIN vendas_por_cliente_temp c ON f.id_cliente = c.id_cliente
''')

┌───────────┬───────────────┬──────────┐
│ anomesdia │ nome_cliente  │ vl_total │
│   int32   │    varchar    │  double  │
├───────────┼───────────────┼──────────┤
│  20240620 │ Yuri Teixeira │    32.64 │
│  20240620 │ Yuri Teixeira │   423.35 │
│  20240620 │ Yuri Teixeira │    32.64 │
│  20240620 │ Yuri Teixeira │   423.35 │
│  20240620 │ Yuri Teixeira │    32.64 │
│  20240620 │ Yuri Teixeira │   423.35 │
│  20240624 │ Thiago Castro │  2470.35 │
│  20240624 │ Thiago Castro │   137.48 │
│  20240624 │ Thiago Castro │   111.35 │
│  20240624 │ Thiago Castro │  2470.35 │
│      ·    │  ·            │      ·   │
│      ·    │  ·            │      ·   │
│      ·    │  ·            │      ·   │
│  20250509 │ NULL          │   149.06 │
│  20250512 │ NULL          │    447.4 │
│  20250512 │ NULL          │   475.31 │
│  20250512 │ NULL          │    447.4 │
│  20250512 │ NULL          │   475.31 │
│  20250512 │ NULL          │    447.4 │
│  20250512 │ NULL          │   475.31 │
│  20250529 │ NU

---

### 3. `RIGHT JOIN` (ou `RIGHT OUTER JOIN`) – Todos da direita

Retorna **todos os registros da tabela da direita** e os correspondentes da esquerda.  
Onde não houver correspondência, os campos da esquerda vêm como `NULL`.

🔍 **Menos comum que o `LEFT JOIN`**, mas útil quando a tabela de referência principal está à direita.

In [33]:
duckdb.sql('''
SELECT f.anomesdia, c.nome as nome_cliente, f.valor_total_item as vl_total
FROM fato_vendas f
RIGHT JOIN vendas_por_cliente_temp c ON f.id_cliente = c.id_cliente
''')

┌───────────┬───────────────┬────────────────────┐
│ anomesdia │ nome_cliente  │      vl_total      │
│   int32   │    varchar    │       double       │
├───────────┼───────────────┼────────────────────┤
│  20240620 │ Yuri Teixeira │              32.64 │
│  20240620 │ Yuri Teixeira │             423.35 │
│  20240620 │ Yuri Teixeira │              32.64 │
│  20240620 │ Yuri Teixeira │             423.35 │
│  20240620 │ Yuri Teixeira │              32.64 │
│  20240620 │ Yuri Teixeira │             423.35 │
│  20240624 │ Thiago Castro │            2470.35 │
│  20240624 │ Thiago Castro │             137.48 │
│  20240624 │ Thiago Castro │             111.35 │
│  20240624 │ Thiago Castro │            2470.35 │
│      ·    │      ·        │               ·    │
│      ·    │      ·        │               ·    │
│      ·    │      ·        │               ·    │
│  20250604 │ Levi Correia  │            1674.35 │
│  20250608 │ Brenda Fogaça │ 1135.1100000000001 │
│  20250608 │ Brenda Fogaça │  

---

### 4. `FULL JOIN` (ou `FULL OUTER JOIN`) – União total

Retorna **todos os registros de ambas as tabelas**, combinando os correspondentes.  
Onde não houver correspondência, os campos da outra tabela serão `NULL`.

🔍 **Útil para** identificar registros que existem apenas em uma das tabelas.

In [34]:
duckdb.sql('''
SELECT f.anomesdia, c.nome as nome_cliente, f.valor_total_item as vl_total
FROM fato_vendas f
FULL OUTER JOIN vendas_por_cliente_temp c ON f.id_cliente = c.id_cliente
''')

┌───────────┬───────────────┬──────────┐
│ anomesdia │ nome_cliente  │ vl_total │
│   int32   │    varchar    │  double  │
├───────────┼───────────────┼──────────┤
│  20240620 │ Yuri Teixeira │    32.64 │
│  20240620 │ Yuri Teixeira │   423.35 │
│  20240620 │ Yuri Teixeira │    32.64 │
│  20240620 │ Yuri Teixeira │   423.35 │
│  20240620 │ Yuri Teixeira │    32.64 │
│  20240620 │ Yuri Teixeira │   423.35 │
│  20240624 │ Thiago Castro │  2470.35 │
│  20240624 │ Thiago Castro │   137.48 │
│  20240624 │ Thiago Castro │   111.35 │
│  20240624 │ Thiago Castro │  2470.35 │
│      ·    │  ·            │      ·   │
│      ·    │  ·            │      ·   │
│      ·    │  ·            │      ·   │
│  20250509 │ NULL          │   149.06 │
│  20250512 │ NULL          │    447.4 │
│  20250512 │ NULL          │   475.31 │
│  20250512 │ NULL          │    447.4 │
│  20250512 │ NULL          │   475.31 │
│  20250512 │ NULL          │    447.4 │
│  20250512 │ NULL          │   475.31 │
│  20250529 │ NU

---

### 5. `CROSS JOIN` – Produto cartesiano

O `CROSS JOIN` combina **cada linha da tabela A com todas as linhas da tabela B**.  
Ele **não precisa de uma condição de junção (`ON`)**, e o resultado será um **produto cartesiano**.

📌 **Exemplo prático:**  
Se a tabela `fato_vendas` tiver 100 linhas e `vendas_por_cliente` tiver 50 linhas,  
o `CROSS JOIN` retornará **5.000 combinações** (100 × 50).

🔍 **Use com cuidado.** Ideal para gerar combinações entre conjuntos de dados pequenos.

In [42]:
duckdb.sql('''
SELECT count(*) AS total_registros
FROM fato_vendas f
CROSS JOIN vendas_por_cliente_temp c
LIMIT 1000
''')

┌─────────────────┐
│ total_registros │
│      int64      │
├─────────────────┤
│           12000 │
└─────────────────┘

In [41]:
duckdb.sql('''
SELECT 
    f.anomesdia, 
    c.nome AS nome_cliente, 
    f.valor_total_item AS vl_total
FROM fato_vendas f
CROSS JOIN vendas_por_cliente_temp c
LIMIT 400
''')

┌───────────┬────────────────┬──────────┐
│ anomesdia │  nome_cliente  │ vl_total │
│   int32   │    varchar     │  double  │
├───────────┼────────────────┼──────────┤
│  20240620 │ Yago Novaes    │    32.64 │
│  20240620 │ Yago Novaes    │   423.35 │
│  20240620 │ Yago Novaes    │    32.64 │
│  20240620 │ Yago Novaes    │   423.35 │
│  20240620 │ Yago Novaes    │    32.64 │
│  20240620 │ Yago Novaes    │   423.35 │
│  20240624 │ Yago Novaes    │  2470.35 │
│  20240624 │ Yago Novaes    │   137.48 │
│  20240624 │ Yago Novaes    │   111.35 │
│  20240624 │ Yago Novaes    │  2470.35 │
│      ·    │      ·         │      ·   │
│      ·    │      ·         │      ·   │
│      ·    │      ·         │      ·   │
│  20241001 │ Isadora Castro │   360.21 │
│  20241001 │ Isadora Castro │   360.21 │
│  20241001 │ Isadora Castro │   360.21 │
│  20241011 │ Isadora Castro │   2009.7 │
│  20241011 │ Isadora Castro │  2367.85 │
│  20241011 │ Isadora Castro │   240.45 │
│  20241011 │ Isadora Castro │  23

### 🔍 O que esse `CROSS JOIN` faz:

- Para **cada venda (`f`)**, ele combina com **cada cliente (`c`)**, mesmo que não estejam relacionados.
- `LIMIT 100` foi adicionado no exemplo para evitar um volume excessivo de linhas.

### 🚫 Quando **não** usar `CROSS JOIN`:

- Quando você **deseja unir tabelas com base em um relacionamento**, como:

```sql
f.id_cliente = c.id_cliente


---

### 📝 Dica prática

| Tipo de JOIN   | Inclui registros de...           | NULLs possíveis em...             |
|----------------|----------------------------------|-----------------------------------|
| `INNER JOIN`   | Ambas as tabelas                 | Nenhuma                           |
| `LEFT JOIN`    | Tabela da esquerda               | Tabela da direita                 |
| `RIGHT JOIN`   | Tabela da direita                | Tabela da esquerda                |
| `FULL JOIN`    | Ambas (todos os registros)       | Ambas                             |
| `CROSS JOIN`   | Produto cartesiano               | Nenhuma (mas volume alto)         |

---

### 💬 Em resumo

> `JOIN` é essencial para integrar tabelas e montar consultas completas em bancos relacionais.  
> Escolher o tipo certo de `JOIN` depende da lógica de negócio e da análise que você quer realizar.

---

# 🔷 O que é DeltaTable?

## 📌 Conceito

`DeltaTable` é o formato de armazenamento criado pelo **Delta Lake**, que adiciona **transações ACID, versionamento e atualizações** ao formato **Parquet**.

Ele combina a **performance do Parquet** com recursos de **banco de dados transacional**, tornando possível fazer `UPDATE`, `DELETE`, `MERGE` e `UPSERT` diretamente em dados armazenados no Data Lake.

---

## 🧩 Estrutura de uma Delta Table

Uma Delta Table é composta por:

- Arquivos **Parquet**: onde os dados reais estão armazenados.
- Pasta `_delta_log/`: contém os **logs de transações**, esquema da tabela e versionamento.

> 📁 Exemplo de estrutura:

fato_vendas_delta/  
├── part-00001.parquet  
├── part-00002.parquet  
└── _delta_log/  
    ├── 00000000000000000000.json  
    ├── 00000000000000000001.json  
    └── ...



---

## 🧠 Por que usar DeltaTable?

| Recurso                     | Parquet | DeltaTable |
|----------------------------|---------|------------|
| Leitura colunar rápida     | ✅      | ✅          |
| Escrita transacional (ACID)| ❌      | ✅          |
| `UPDATE` / `DELETE`        | ❌      | ✅          |
| Time travel (versões)      | ❌      | ✅          |
| Schema evolution           | ❌      | ✅          |

---

## ✨ Benefícios do DeltaTable

- ✅ Suporte a **transações ACID**
- ✅ Possibilidade de **atualizar e deletar dados**
- ✅ **Time travel**: recuperar versões anteriores
- ✅ **Schema evolution**: ajustar esquema de forma automática
- ✅ Compatível com ferramentas como **Spark, Databricks, Presto e AWS Athena (via lakehouse)**


## ✅ Equivalentes ao DuckDB para manipular Delta Table com Spark

Aqui está um guia com equivalentes comuns:

| Operação                   | DuckDB                                       | Delta Lake (Spark)                                                  |
|----------------------------|----------------------------------------------|----------------------------------------------------------------------|
| Leitura (`SELECT`)        | `duckdb.sql("SELECT * FROM fato")`           | `spark.read.format("delta").load("s3://...")`                        |
| `UPDATE`                  | `UPDATE fato SET ...`                        | `DeltaTable.forPath(...).update(...)`                               |
| `DELETE`                  | `DELETE FROM fato WHERE ...`                 | `DeltaTable.forPath(...).delete(...)`                               |
| `TRUNCATE` (`DELETE all`) | `DELETE FROM fato`                           | `DeltaTable.forPath(...).delete("true")`                            |
| `INSERT INTO`             | `INSERT INTO fato VALUES (...)`             | `df.write.format("delta").mode("append").save(...)`                 |
| `MERGE INTO` (upsert)     | ❌ (não suportado)                            | `DeltaTable.forPath(...).merge(...).whenMatchedUpdate().execute()` |

---

## 🔧 Exemplo prático com Spark + Delta Lake



In [ ]:
%pip install pyspark delta-spark

In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

builder = SparkSession.builder \
    .appName("Delta + S3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")

spark = configure_spark_with_delta_pip(builder).getOrCreate()


# Inicializa Spark com suporte ao Delta
spark = configure_spark_with_delta_pip(builder).getOrCreate()



In [ ]:
# 2. Caminhos no S3
parquet_path = "s3a://aula-data-lake/gold/fato_vendas/anomesdia=20240620/"
delta_path = "s3://aula-data-lake/gold/fato_vendas_delta/"

# 3. Ler os dados Parquet existentes
df = spark.read.parquet(parquet_path)

df.show()

In [ ]:
# 4. Escrever os dados no formato Delta
df.write.format("delta").mode("overwrite").save(delta_path)

# Carregando a Delta Table
delta_table = DeltaTable.forPath(spark, delta_path)

In [ ]:
delta_table.select("*").show(10)

### ✏️ UPDATE: marcar pendentes como cancelado

In [ ]:
delta_table.update(
    condition="status = 'PENDENTE'",
    set={"status": "'CANCELADO'"}
)

In [ ]:
spark.sql(f"""
    UPDATE delta.`{delta_path}`
    SET status = 'CANCELADO'
    WHERE status = 'PENDENTE'
""")

### 🗑️ DELETE: remover preços baixos

In [ ]:
delta_table.delete("preco_unitario < 1")

In [ ]:
spark.sql(f"""
    DELETE FROM delta.`{delta_path}`
    WHERE preco_unitario < 1
""")

### 🧹 TRUNCATE: remover tudo


In [ ]:
delta_table.delete("true")  # equivalente ao TRUNCATE

In [ ]:
spark.sql(f"""
    DELETE FROM delta.`{delta_path}`
    WHERE true
""")

### 📊 SELECT (como no DuckDB)

In [ ]:
df = spark.read.format("delta").load(delta_path)
df.select("nome_produto", "quantidade").show()

### ⚠️ Dica: usar SQL direto no Delta com Spark

In [ ]:
spark.sql(f"""
    SELECT nome_tipo, SUM(valor_total_item) 
    FROM delta.`{delta_path}`
    GROUP BY nome_tipo
""").show()


### 📝 Observação

Você também pode registrar a Delta Table como uma **tabela permanente no metastore** para usar nomes simples:

In [ ]:
# Registrando a tabela
spark.sql(f"""
    CREATE TABLE fato_vendas_delta
    USING DELTA
    LOCATION '{delta_path}'
""")

# Agora pode fazer:
spark.sql("UPDATE fato_vendas_delta SET status = 'CANCELADO' WHERE status = 'PENDENTE'")

### ✅ Conclusão

| Você quer...                                      | Ferramenta recomendada   |
|--------------------------------------------------|---------------------------|
| Manipular dados locais ou Parquet simples        | DuckDB ou Pandas          |
| Manipular Delta Tables com controle transacional | Spark + Delta Lake        |
